## 使用DeepSeek进行数据表的查询

In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

db_user = "student123"
db_password = "student321"
#db_host = "localhost:3306"
db_host = "rm-uf6z891lon6dxuqblqo.mysql.rds.aliyuncs.com:3306"
db_name = "action"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
db

In [2]:
from langchain.chat_models import ChatOpenAI
import os

# 从环境变量获取 dashscope 的 API Key
api_key = os.environ.get('DASHSCOPE_API_KEY')

# 通过LLM => 撰写SQL
llm = ChatOpenAI(
    temperature=0.01,
    model="deepseek-v3",  
    #model = "qwen-turbo",
    # openai_api_key = "sk-9846f14a2104490b960adbf5c5b3b32e",
    # openai_api_base="https://api.deepseek.com"
    openai_api_base = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_api_key  = api_key
)

# 需要设置llm
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# SQL智能体：给它目标，它自己会进行规划，最终把结果给你
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

C:\Users\cheny\AppData\Local\Temp\ipykernel_15276\3561425379.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [3]:
# Task: 描述数据表
agent_executor.run("描述与订单相关的表及其关系")

C:\Users\cheny\AppData\Local\Temp\ipykernel_15276\2182499378.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("描述与订单相关的表及其关系")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""address, asset_grades, bank, car_sales, customers, dept, employee, form, height_grades, hero_score, heros, orders, person, player, player_score, student, team, team_score, test_work, trips, user, users, using, weatherI see that there is an "orders" table which is likely related to orders. I should also check if there are other tables that might be related to orders, such as "customers" or "users". I will query the schema of these tables to understand their structure and relationships.

Action: sql_db_schema
Action Input: "orders, customers, users"
CREATE TABLE customers (
	`Id` INTEGER, 
	`Name` VARCHAR(255)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb3

/*
3 rows from customers table:
Id	Name
1	Joe
2	Henry
3	Sam
*/


CREATE TABLE orders (
	`Id` INTEGER, 
	`CustomerId` INTEGER
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb3

/*
3 rows from orders table:
Id	CustomerId
1	3
2	1
*/


CREATE TABLE users (
	`Users_Id` IN

'与订单相关的表主要是orders表和customers表，它们之间的关系是：orders表中的CustomerId字段作为外键，关联到customers表的Id主键。这种关系表示一个客户可以有多个订单（一对多关系）。users表虽然存在，但当前模式中没有显示与orders表的直接关联。'

In [4]:
# 这个任务，实际上数据库中 没有HeroDetails表
agent_executor.run("描述HeroDetails表")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""address, asset_grades, bank, car_sales, customers, dept, employee, form, height_grades, hero_score, heros, orders, person, player, player_score, student, team, team_score, test_work, trips, user, users, using, weatherI see that there's a table called "heros" which might be related to "HeroDetails". I'll check the schema of this table to understand its structure.

Action: sql_db_schema
Action Input: "heros"
CREATE TABLE heros (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL, 
	hp_max FLOAT, 
	hp_growth FLOAT, 
	hp_start FLOAT, 
	mp_max FLOAT, 
	mp_growth FLOAT, 
	mp_start FLOAT, 
	attack_max FLOAT, 
	attack_growth FLOAT, 
	attack_start FLOAT, 
	defense_max FLOAT, 
	defense_growth FLOAT, 
	defense_start FLOAT, 
	hp_5s_max FLOAT, 
	hp_5s_growth FLOAT, 
	hp_5s_start FLOAT, 
	mp_5s_max FLOAT, 
	mp_5s_growth FLOAT, 
	mp_5s_start FLOAT, 
	attack

'The "heros" table (which appears to be the closest match to "HeroDetails") contains detailed information about heroes with the following columns:\n- Basic identifiers: id, name\n- Health points (HP) statistics: hp_max, hp_growth, hp_start\n- Magic points (MP) statistics: mp_max, mp_growth, mp_start\n- Attack statistics: attack_max, attack_growth, attack_start\n- Defense statistics: defense_max, defense_growth, defense_start\n- Regeneration stats: hp_5s_max, hp_5s_growth, hp_5s_start, mp_5s_max, mp_5s_growth, mp_5s_start\n- Combat attributes: attack_speed_max, attack_range\n- Role information: role_main, role_assist\n- Birthdate\n\nThe table stores comprehensive game character attributes including growth rates, maximum values, and starting values for various combat metrics. Sample heroes include "夏侯惇" (Xiahou Dun), "钟无艳" (Zhong Wuyan), and "钟" (Zhong).'

In [5]:
agent_executor.run("描述Hero表")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: address, asset_grades, bank, car_sales, customers, dept, employee, form, height_grades, hero_score, heros, orders, person, player, player_score, student, team, team_score, test_work, trips, user, users, using, weatherI see that there are tables named "heros" and "hero_score" which might be relevant to the question about a "Hero" table. I should check the schema of these tables to understand their structure.

Action: sql_db_schema
Action Input: heros, hero_score
CREATE TABLE hero_score (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	score INTEGER NOT NULL DEFAULT '0', 
	name VARCHAR(20) CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci, 
	PRIMARY KEY (id)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from hero_score table:
id	score	name
1	80	张飞
2	95	关羽
3	92	刘备
*/


CREATE TABLE heros (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8 COLLATE utf8_genera

'The "heros" table contains detailed information about heroes with the following columns:\n- id (primary key)\n- name (hero name)\n- Various hero attributes including:\n  - hp_max, hp_growth, hp_start (health points)\n  - mp_max, mp_growth, mp_start (magic points)\n  - attack_max, attack_growth, attack_start (attack stats)\n  - defense_max, defense_growth, defense_start (defense stats)\n  - hp_5s_max, hp_5s_growth, hp_5s_start (health regeneration)\n  - mp_5s_max, mp_5s_growth, mp_5s_start (magic regeneration)\n  - attack_speed_max\n  - attack_range\n  - role_main and role_assist (hero roles)\n  - birthdate\n\nSample heroes include 夏侯惇 (Xiahou Dun), 钟无艳 (Zhong Wuyan), and 钟 (Zhong).\n\nThe "hero_score" table is simpler, containing:\n- id (primary key)\n- score\n- name (hero name)\n\nSample data shows scores for 张飞 (Zhang Fei), 关羽 (Guan Yu), and 刘备 (Liu Bei).'

In [6]:
agent_executor.run("找出英雄攻击力最高的前5个英雄")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: address, asset_grades, bank, car_sales, customers, dept, employee, form, height_grades, hero_score, heros, orders, person, player, player_score, student, team, team_score, test_work, trips, user, users, using, weatherI see there are tables named "heros" and "hero_score" which are likely relevant to the question about heroes' attack power. I should check the schema of these tables to understand their structure.

Action: sql_db_schema
Action Input: heros, hero_score
CREATE TABLE hero_score (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	score INTEGER NOT NULL DEFAULT '0', 
	name VARCHAR(20) CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci, 
	PRIMARY KEY (id)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from hero_score table:
id	score	name
1	80	张飞
2	95	关羽
3	92	刘备
*/


CREATE TABLE heros (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8 COLLATE utf8_gen

'攻击力最高的前5个英雄是：阿轲(427)、孙尚香(411)、百里守约(410)、虞姬(407)、黄忠(403)。'

In [9]:
#help(agent_executor.run)